## Init

**WARNING: For the server, only tensorflow 1.x is supported!!**


In [2]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Requirement already up-to-date: bert-serving-server[http] in /home/salvi/.local/lib/python3.7/site-packages (1.10.0)


In [1]:
import os

from bert_serving.client import BertClient

In [12]:
init = 1

# Large: https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
# Base: https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
link = (
    "https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip"
)
path = "/scratch/WikipediaImagesTaxonomy/embeddings/"
curr_path = os.getcwd()

if init:

    os.chdir(path)
    !wget {link}
    !unzip {link.split("/")[-1].split(".")[0]}
    !rm {link.split("/")[-1]}
    os.chdir(curr_path)

--2022-10-25 20:32:16--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.215.240, 172.217.168.48, 172.217.168.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.215.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: 'uncased_L-12_H-768_A-12.zip'

uncased_L-12_H-768_ 100%[===================>] 388.84M   104MB/s    in 3.8s    

2022-10-25 20:32:20 (104 MB/s) - 'uncased_L-12_H-768_A-12.zip' saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [15]:
# !nohup bert-serving-start -model_dir={path + link.split("/")[-1].split(".")[0]} -max_seq_len=512 > bert_server_log.file 2>&1 &

# Command to execute in an environment with tensorflow 1.x
print(
    f'nohup bert-serving-start -model_dir={path + link.split("/")[-1].split(".")[0]} -max_seq_len=512 > bert_server.log 2>&1 &'
)

nohup bert-serving-start -model_dir=/scratch/WikipediaImagesTaxonomy/embeddings/uncased_L-12_H-768_A-12 -max_seq_len=512 > bert_server.log 2>&1 &


## Test

In [22]:
bc = BertClient()

In [17]:
sentence1 = """The Coca-Cola Company's formula for Coca-Cola syrup, which bottlers combine with carbonated 
water to create the company's flagship cola soft drink, is a closely guarded trade secret."""
sentence2 = """Dr Pepper is a carbonated soft drink. It was created in the 1880s by pharmacist Charles Alderton in Waco, 
Texas, and first served around 1885."""
sentence3 = """A vector space (also called a linear space) is a set of objects called vectors, 
which may be added together and multiplied ("scaled") by numbers, called scalars."""
sentence4 = """In mathematics, specifically abstract algebra, an integral domain is a 
nonzero commutative ring in which the product of any two nonzero elements is nonzero."""

vectors = bc.encode([sentence1, sentence2, sentence3, sentence4])
vectors

array([[ 0.21223839,  0.3963629 ,  0.18777731, ..., -0.34814426,
         0.01989033,  0.2920837 ],
       [ 0.00231424,  0.32675627, -0.16526939, ..., -0.28342417,
        -0.08130819,  0.2145305 ],
       [ 0.27074087, -0.02507309,  0.34614998, ..., -0.3749927 ,
        -0.65026706,  1.1105642 ],
       [-0.06408317, -0.12897359,  0.17251775, ..., -0.33599234,
        -0.7614141 ,  1.1481148 ]], dtype=float32)

In [18]:
from scipy import spatial

for a in range(len(vectors)):
    for b in range(a, len(vectors)):
        print(
            "Distance between sentence{} and sentence{} == {}".format(
                a + 1, b + 1, spatial.distance.cosine(vectors[a], vectors[b])
            )
        )

Distance between sentence1 and sentence1 == 0
Distance between sentence1 and sentence2 == 0.21456605195999146
Distance between sentence1 and sentence3 == 0.4746311902999878
Distance between sentence1 and sentence4 == 0.6466650664806366
Distance between sentence2 and sentence2 == 0
Distance between sentence2 and sentence3 == 0.48074793815612793
Distance between sentence2 and sentence4 == 0.6145219206809998
Distance between sentence3 and sentence3 == 0
Distance between sentence3 and sentence4 == 0.20339584350585938
Distance between sentence4 and sentence4 == 0


In [23]:
sentence1 = """Nature"""
sentence2 = """Society"""
sentence3 = """Plants"""
sentence4 = """History"""

vectors = bc.encode([sentence1, sentence2, sentence3, sentence4])
vectors

array([[-0.3564243 , -0.38662854, -0.33100328, ..., -0.02123652,
        -0.01991034,  0.22340636],
       [-0.45478043, -0.5048582 , -0.43433663, ..., -0.17134349,
        -0.12261462,  0.18812723],
       [-0.14301233,  0.44115624,  0.01763745, ..., -0.17991972,
        -0.4929944 , -0.12789269],
       [-0.13638987,  0.01003768, -0.48937917, ..., -0.37142923,
        -0.14123178,  0.08776691]], dtype=float32)

In [24]:
from scipy import spatial

for a in range(len(vectors)):
    for b in range(a, len(vectors)):
        print(
            "Distance between sentence{} and sentence{} == {}".format(
                a + 1, b + 1, spatial.distance.cosine(vectors[a], vectors[b])
            )
        )

Distance between sentence1 and sentence1 == 0
Distance between sentence1 and sentence2 == 0.20644283294677734
Distance between sentence1 and sentence3 == 0.3565189242362976
Distance between sentence1 and sentence4 == 0.3293205499649048
Distance between sentence2 and sentence2 == 0
Distance between sentence2 and sentence3 == 0.36850881576538086
Distance between sentence2 and sentence4 == 0.3361852765083313
Distance between sentence3 and sentence3 == 0
Distance between sentence3 and sentence4 == 0.3378022313117981
Distance between sentence4 and sentence4 == 0


In [25]:
bc.close()